## Analysis of my AWS costs for the project

In [ ]:
import pandas as pd

inst = pd.read_csv('../../blog-post/instance-costs.csv', parse_dates=True)
inst = inst.iloc[1:, :] # drop total col

inst = inst.fillna(0)

inst

In [ ]:
summary = inst.sum(axis=0).to_frame()

summary = summary.iloc[1:, :]
summary.columns = ['Cost [$]']

summary.index = [str(i)[:-3] for i in summary.index]

metadata = pd.DataFrame({
    'm5.16xlarge': ['controller', 64, 256, 0],
    'p3.2xlarge': ['vae-and-memory', 8, 61, 16],
    'p2.xlarge': ['vae-and-memory', 4, 61, 12],
    
    'c5.2xlarge': ['sample-experience', 8, 16, 0],
    'c5.4xlarge': ['sample-experience', 16, 32, 0],
    
    'm4.16xlarge': ['sample-latent-stats', 16, 64, 0],
    
    't2.xlarge': ['misc', 4, 16, 0]

}).T

metadata.columns = ['component', 'cpu [num]', 'memory [GB]', 'gpu [GB]']

summary = summary.drop('No Instance Type', axis=0)
summary = summary.drop('Total cost ', axis=0)

summary

In [ ]:
metadata.head()

In [ ]:
out = pd.concat([metadata, summary], axis=1, sort=False).sort_values('Cost [$]', ascending=False)

out = out.dropna(axis=0)

out

In [ ]:
out.groupby('component').sum().sort_values('Cost [$]', ascending=False)

In [ ]:
# want per component, per month!

In [ ]:
inst

In [ ]:
metadata

In [ ]:
from collections import defaultdict
component_per_month = defaultdict(list)

for instance in metadata.index:
    component = metadata.loc[instance, 'component']
    component_per_month[component].append(inst.loc[:, instance+'($)'])

for k, v in component_per_month.items():
    component_per_month[k] = sum(v)
    
component_per_month = pd.DataFrame(component_per_month)
component_per_month.index = inst.loc[:, 'InstanceType']

component_per_month

In [ ]:
component_per_month.plot(kind='bar')

In [ ]:
serv = pd.read_csv('../blog-post/service-costs.csv')
serv = serv.iloc[1:, :]

serv = serv.fillna(0)

serv = serv.set_index('Service', drop=True)
serv

In [ ]:
per_serv = serv.sum(axis=0).to_frame()
per_serv = per_serv.iloc[1:, :]

per_serv

In [ ]:
# put agent numbers on here
month = serv.sum(axis=1).to_frame()

month.index = serv.index

month.plot(kind='bar')